In [0]:
!pip install fastai
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html

    100% |████████████████████████████████| 17.3MB 1.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/48/5d/04/22361a593e70d23b1f7746d932802efe1f0e523376a74f321e
Successfully built wrapt
torchvision 0.2.2.post2 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
pymc3 3.6 has requirement joblib<0.13.0, but you'll have joblib 0.13.2 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6
  Found existing installation: wrapt 1.11.1
    Uninstalling wrapt-1.11.1:
      Successfully uninstalled wrapt-1.11.1


Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
    100% |████████████████████████████████| 576.2MB 23kB/s 


In [0]:
import numpy as np 
import pandas as pd 
from fastai import *
from fastai.text import *

In [0]:
poem = pd.read_csv('../content/all.csv')
print(poem.head())
print(poem.tail())


                                    author  \
0                      WILLIAM SHAKESPEARE   
1  DUCHESS OF NEWCASTLE MARGARET CAVENDISH   
2                           THOMAS BASTARD   
3                           EDMUND SPENSER   
4                        RICHARD BARNFIELD   

                                             content  \
0  Let the bird of loudest lay\r\nOn the sole Ara...   
1  Sir Charles into my chamber coming in,\r\nWhen...   
2  Our vice runs beyond all that old men saw,\r\n...   
3  Lo I the man, whose Muse whilome did maske,\r\...   
4  Long have I longd to see my love againe,\r\nSt...   

                                 poem name          age                  type  
0               The Phoenix and the Turtle  Renaissance  Mythology & Folklore  
1                 An Epilogue to the Above  Renaissance  Mythology & Folklore  
2                       Book 7, Epigram 42  Renaissance  Mythology & Folklore  
3  from The Faerie Queene: Book I, Canto I  Renaissance  Mythology

In [0]:
poem['content'] = poem['content'].str.lower()
print(poem.head())

                                    author  \
0                      WILLIAM SHAKESPEARE   
1  DUCHESS OF NEWCASTLE MARGARET CAVENDISH   
2                           THOMAS BASTARD   
3                           EDMUND SPENSER   
4                        RICHARD BARNFIELD   

                                             content  \
0  let the bird of loudest lay\r\non the sole ara...   
1  sir charles into my chamber coming in,\r\nwhen...   
2  our vice runs beyond all that old men saw,\r\n...   
3  lo i the man, whose muse whilome did maske,\r\...   
4  long have i longd to see my love againe,\r\nst...   

                                 poem name          age                  type  
0               The Phoenix and the Turtle  Renaissance  Mythology & Folklore  
1                 An Epilogue to the Above  Renaissance  Mythology & Folklore  
2                       Book 7, Epigram 42  Renaissance  Mythology & Folklore  
3  from The Faerie Queene: Book I, Canto I  Renaissance  Mythology

In [0]:
test_per = 0.05        # test percentage
poem = poem.iloc[np.random.permutation(len(poem))]
cut = int(test_per * len(poem)) + 1
train_poem, test_poem = poem[cut:], poem[:cut]

In [0]:
print(len(train_poem),len(test_poem))

544 29


In [0]:
# Pre-processing
nan_row = poem[poem['content'].isnull()]
print(nan_row)

Empty DataFrame
Columns: [author, content, poem name, age, type]
Index: []


In [0]:
data_lm = TextLMDataBunch.from_df('content', train_poem, test_poem, text_cols = 'content', label_cols = 'author')
data_clas = TextClasDataBunch.from_df('content', train_poem, test_poem, vocab = data_lm.train_ds.vocab, bs = 32)
#data_lm.save('data_lm_export.pkl')
#data_clas.save('data_clas_export.pkl')

/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:498: UserWarning: You are labelling your items with CategoryList.
Your valid set contained the following unknown labels, the corresponding items have been discarded.
ORLANDO GIBBONS
  if getattr(ds, 'warn', False): warn(ds.warn)


In [0]:
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult = 0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
1,6.675148,5.750742,0.152902,06:06


In [0]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
1,5.602635,4.839277,0.236719,09:29


In [0]:
wd=1e-7
lr=1e-3
lrs = lr

In [0]:
learn.fit(10,lrs, wd)

epoch,train_loss,valid_loss,accuracy,time
1,5.084971,4.625491,0.247098,09:14
2,4.878168,4.528723,0.253906,09:20
3,4.703879,4.468982,0.255357,09:15
4,4.526164,4.456409,0.257478,09:16
5,4.350982,4.445903,0.256696,09:17
6,4.164670,4.427321,0.254799,09:16
7,3.979441,4.458419,0.250446,09:13
8,3.798104,4.463895,0.252455,09:28
9,3.602679,4.462164,0.250223,09:38
10,3.421247,4.547796,0.248772,09:35


In [0]:
learn.predict("xxbos", n_words=150, temperature=0.75)

'xxbos i see two knights from each side : \r \n  to whom i a great prince of kings , \r \n  a lord of great great fortune , \r \n  beseeming with gifts of wit , \r \n  a peer , and a great prince , \r \n  till the latest day did not so well \r \n  \r \n  with a continual change , though they knew not what he was : \r \n  but when he came to thee , he made his happiest day , \r \n  which now did not last ; \r \n  his absence was one that might him lead . \r \n \r \n  but when , in terror , how many a great prince did he \r \n  \r \n  a man of good fortune , he died \r \n  he felt the years follow upon him , \r \n  but that he was not so short of care , \r \n  nor looked , nor in mind , \r \n '

In [0]:
number_of_poems = 15
poems_counter = 0
all_poems = []

for i in range(1000):
    poem = learn.predict("xxbos ", n_words=70, temperature=0.8)
    poems = poem.split("xxbos ")
    poems = poems[1:-1]
    
    for poem in poems:
        poem = poem.replace("xxbos ","").strip()
        if(poem):
            all_poems.append(poem)
            poems_counter = poems_counter+1
            
    if poems_counter > number_of_poems:
        break

In [0]:
all_poems

['the cross and the cross and me \r \n  are thee a - might hold , \r \n  and a good one are best . \r \n  love should not be of love , \r \n  but that for love , love .',
 'whether either were there , or not , what have you done , how guilty , what is your right , \r \n  why do you make your great change ? \r \n  whether that be rights vacant or died .',
 'the songs i from my songs pass \r \n  from the wild poems of poets . copyright 1993 by ezra pound . reprinted by permission of new directions publishing corporation .',
 'some 1928 treason of poet',
 "Tom o ' Of Time , the collected poems of poet george p. yeats , edited by george j. firmage . copyright 1933 , 1951 , 1954 by wallace stevens . used by permission of asphodel press / acorn alliance .",
 'archibald macleish , the golden cross from selected poems , published by harpercollins publishers australia . used by permission .',
 'kenneth slessor , surge from selected poems , published by harpercollins publishers australia . used 

In [0]:
outF = open("../content/poem.txt", "w")
for poem_a in all_poems:
  # write line to output file
  outF.write(poem_a)
  outF.write("\n***********************************************************************************\n")
outF.close()

In [27]:
learn.predict("xxbos xxfld 1", n_words=35, temperature=0.8)

'xxbos xxfld 1 , 1917 with the permission of a. p. watt , ltd . on behalf of michael yeats . xxbos when i have heard the rain slide through my heart , \r \n  i once again heard'

In [0]:
number_of_poems1 = 30
poems_counter1 = 0
all_poems1 = []

for i in range(1000):
    poem1 = learn.predict("xxbos ", n_words=150, temperature=0.8)
    poems1 = poem1.split("xxbos ")
    poems1 = poems1[1:-1]
    
    for poem1 in poems1:
      if ("copyright" or "published") not in poem1:
        poem1 = poem1.replace("xxbos ","").strip()
        if(poem1):
            all_poems1.append(poem1)
            poems_counter1 = poems_counter1+1
            
    if poems_counter1 > number_of_poems1:
        break

In [35]:
all_poems1

['be beyond your reach , and be my love , \r \n  for thus i love my love , that i love . \r \n  but if i take too much faith , \r \n  as a child i may not love my love , \r \n  yet will leave my love as love in love .',
 'my love is not always fragile , \r \n  but the heart is grown \r \n  of a grace to which she is many : \r \n  and for all the reason her mind is owne , \r \n  the heart of my eternity doth stand , \r \n  to be resurrection , and to allow it to be .',
 "t '",
 "michael anania , fall from complete poems : 1954 , 1954 by random house , 1958 . reprinted with the permission of farrar , straus & giroux , llc , http : / / us.macmillan.com / fsg . all rights 'll be reserved .",
 'all the day it had been spread , \r \n  and all the more , the only thing \r \n  which we are great , \r \n  would not have worth a corner to lose ; \r \n  but when we over the clock \r \n  they catch the little birds \r \n  that are the next reason you love you .',
 'originally published in poetry ,

In [0]:
outF = open("../content/poem_more.txt", "w")
for poem_a in all_poems:
  # write line to output file
  outF.write(poem_a)
  outF.write("\n*******************************************************************************************\n")
outF.close()

In [0]:
number_of_poems1 = 30
poems_counter1 = 0
all_poems2 = []

for i in range(1000):
    poem = learn.predict("xxbos ", n_words=150, temperature=0.82)
    poems1 = poem1.split("xxbos ")
    poems1 = poems1[1:-1]
    
    for poem1 in poems1:
      if ("copyright" or "published" or "publishers" or "inc" or "originally") not in poem1:
        poem1 = poem1.replace("xxbos ","").strip()
        if(poem1):
            all_poems1.append(poem1)
            poems_counter1 = poems_counter1+1
            
    if poems_counter1 > number_of_poems1:
        break

In [38]:
all_poems1

['an angel that hath taught me to \r \n  often wept with woe , and carelesse dear sympathy \r \n  with a strong rhyme to the end , \r \n  a lovely child , a lovely boy , \r \n  to joy , a joy , to feel , \r \n  to love , and care to be . \r \n \r \n  who , as he before he won , \r \n  his bodies were not their worth : \r \n  his body chorus were adorned with \r \n  eke i must have , \r \n  long since the first dark of a day \r \n  did grow in vain . \r \n \r \n  the earth still things stand , \r \n  and the deaths - love thee \r \n  storms , whose deaths are now eternal wars , \r \n  strive no more for those lovers deaths .',
 "go down the roadway , breaking through a bush , \r \n  and looking like the hero , naught as a young maiden , \r \n  so momus , full of fear and shame , \r \n  that i in his meek lie lonely in the dark . \r \n  then build a map of heaven by ; \r \n  you see me , that 's not right , \r \n  the dark skies , \r \n  the fields where are the grass \r \n  and the sky 

In [0]:
outF = open("../content/better_poems.txt", "w")
j = 0
for poem_a in all_poems1:
  # write line to output file
  j=j+1
  outF.write(poem_a)
  outF.write("\n "+str(j)+" \n*********************************************************************************************\n\n")
outF.close()